Использование API, позволяющего получать данные о музейных коллекциях

In [1]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Все произведения, связанные с подсолнухами:

In [2]:
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?q=sunflowers')
res = r.json()

In [3]:
sunflower_ids = res['objectIDs'] 

In [4]:
lst = [437329, 16822570, 2032, 436524, 20149]
for i in lst:
    if i in sunflower_ids:
        print(i)

437329
2032
436524


Информация об объекте 437980:

In [5]:
r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/437980')
obj_test = r_test.json()

In [6]:
lst2 = ['department', 'GalleryNumber', 'culture', 'objectID', 'objectName', 'artistDisplayName', 'title']
for i in lst2:
    print(i, obj_test[i])

department European Paintings
GalleryNumber 825
culture 
objectID 437980
objectName Painting
artistDisplayName Vincent van Gogh
title Cypresses


In [17]:
museum = pd.DataFrame()

Информация об объектах с подсолнухами:

In [18]:
for ids in sunflower_ids:
    try:
        r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{ids}')
        obj_test = r_test.json()
        obj_temp = pd.DataFrame({'objectID' : obj_test['objectID'],
                                  'title' : [obj_test['title']],
                                  'artistDisplayName': [obj_test['artistDisplayName']],
                                  'department' : [obj_test['department']], 
                                  'objectBeginDate' : [obj_test['objectBeginDate']], 
                                  'objectEndDate' : [obj_test['objectEndDate']], 
                                  'period' : [obj_test['period']], 
                                  'culture': [obj_test['culture']]
                          }) 
# приклеиваем
        museum = pd.concat([museum, obj_temp])
    except:
        continue

In [9]:
museum.head()

,objectID,title,artistDisplayName,department,objectBeginDate,objectEndDate,period,culture
0,436524,Sunflowers,Vincent van Gogh,European Paintings,1887,1887,,
0,484935,Sunflowers,Edward McKnight Kauffer,Modern and Contemporary Art,1921,1921,,
0,437112,Bouquet of Sunflowers,Claude Monet,European Paintings,1881,1881,,
0,210191,Vase with sunflowers,Rozenburg Plateelfabriek,European Sculpture and Decorative Arts,1896,1896,,
0,431264,Sunflowers,Lily Converse,Drawings and Prints,1941,1941,,


In [10]:
museum.query("culture == 'Japan'")

,objectID,title,artistDisplayName,department,objectBeginDate,objectEndDate,period,culture
0,57922,Incense box,,Asian Art,1700,1799,Edo period (1615–1868),Japan


Информация о департаменте "Asian art" (departmentId = 6) с тегом "cat" :

In [16]:
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=6&q=cat')
cats = r.json()

In [15]:
cats_ids = cats['objectIDs']

In [20]:
df_cats = pd.DataFrame()
iteration = 0
for ids in cats_ids:
    try:
        r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{ids}')
        obj_test = r_test.json()
        obj_temp = pd.DataFrame({'objectID' : obj_test['objectID'],
                                  'title' : [obj_test['title']],
                                  'artistDisplayName': [obj_test['artistDisplayName']],
                                  'department' : [obj_test['department']], 
                                  'objectBeginDate' : [obj_test['objectBeginDate']], 
                                  'objectEndDate' : [obj_test['objectEndDate']], 
                                  'period' : [obj_test['period']], 
                                  'culture': [obj_test['culture']]
                          }) 
# приклеиваем
        df_cats = pd.concat([df_cats, obj_temp])
        iteration += 1
        if iteration == 1000:
            break
    except:
        continue

In [25]:
df_cats.groupby('period').agg({'title':'count'}).sort_values('title')

,title
period,
"Ming dynasty(1368–1644), Xuande mark and period (1426–35)",1
Muromachi (1392–1573)–Edo period (1615–1868),1
Muromachi (1392–1573),1
Mughal period (1526–1858),1
Western Jin dynasty (265–316),1
...,...
Muromachi period (1392–1573),30
Ming dynasty (1368–1644),49
Qing dynasty (1644–1911),61
